In [1]:
from graphbrain import *
from graphbrain.utils.lemmas import lemma

In [2]:
hg = hgraph('kuznets.hg')

In [3]:
# Read hyperedges

seqs = list(hg.sequences())
edges = []
for seq in hg.sequences():
    for edge in hg.sequence(seq):
        edges.append(edge)

In [6]:
# CLASSES = {'test', 'result', 'positive', 'negative', 'n-shape'}


RESULTS_PREDS = {'show', 'indicate', 'confirm', 'support', 'suggest', 'reveal', 'provide', 'validate', 'exist', 'demonstrate', 'verify', 'imply', 'illustrate', 'find', 'point', 'exhibit', 'establish', 'obtain', 'hold', 'follow'}
SUBJ_OK = {'exist', 'hold'}
NEG_RESULTS_PREDS = {'reject', 'challenge', 'fail'}
TEST_PREDS = {'examine', 'investigate', 'test', 'aim', 'analyze', 'explore', 'assess', 'evaluate_pattern', 'focus'}
RESULT_KEYWORDS = {'result', 'finding', 'test', 'evidence', 'support'}
EKC_KEYWORDS = {'kuznets', 'ekc', 'ekcs', 'ushape', 'ushaped', 'nshape', 'nshaped', 'turning'}
CURVE_KEYWORDS = {'curve', 'shape', 'shaped', 'inverted'}


def predicate_atom(edge):
    if edge.atom:
        if edge.mt == 'P':
            return edge
        return None
    else:
        ct = edge.cmt
        if ct == 'J':
            return predicate_atom(edge[1])
        et = edge.mt
        if et == 'R':
            if ct == 'M':
                return predicate_atom(edge[1])
            else:
                return predicate_atom(edge[0])
        elif et == 'P':
            return predicate_atom(edge[1])
        return None

def contains_keywords(edge, keywords):
    return any((keyword in str(edge)) for keyword in keywords)

def extract_subjects(edge):
    subjs = []
    if edge.atom:
        return subjs
    if edge[0].mt == 'P':
        plemma = lemma(hg, predicate_atom(edge), same_if_none=True).root()
        if plemma not in SUBJ_OK and (plemma in RESULTS_PREDS or plemma in NEG_RESULTS_PREDS):
            subjs += edge.edges_with_argrole('s')
    for subedge in edge[1:]:
        subjs += extract_subjects(subedge)
    return subjs
    
def is_test(edge):
    if edge.type()[0] != 'R':
        return False
    plemma = lemma(hg, predicate_atom(edge), same_if_none=True).root()
    return plemma in TEST_PREDS

def find_test(edge):
    if is_test(edge):
        return edge
    if not edge.is_atom():
        for subedge in edge:
            res = find_test(subedge)
            if res:
                return res
    return None

def is_result(edge):
    if edge.type()[0] != 'R':
        return False

    plemma = lemma(hg, predicate_atom(edge), same_if_none=True).root()
    if plemma not in RESULTS_PREDS and plemma not in NEG_RESULTS_PREDS:
        return False

    # do not allow EKC_KEYWORDS in subject
    subjs = extract_subjects(edge)
    for subj in subjs:
        roots = set(atom.root() for atom in subj.atoms())
        if len(roots & (EKC_KEYWORDS | CURVE_KEYWORDS)) > 0:
            return False
    
    return True

def find_result(edge):
    if is_test(edge):
        return False
    if is_result(edge):
        return edge
    if not edge.is_atom():
        for subedge in edge:
            res = find_result(subedge)
            if res:
                return res
    return None
    
def contains_negator(edge):
    sedge = str(edge)
    neg = '/Mn' in sedge or 'no/M' in sedge or 'little/M' in sedge or 'poor/M' in sedge
    return neg
    
def is_positive(edge, keywords, positive=True):
    if edge.atom:
        root = edge.root()
        if root in keywords:
            return True, positive
        lroot = lemma(hg, edge, same_if_none=True).root()
        if lroot in keywords:
            return True, positive
        return False, positive
    
    sedge = str(edge[0])
    found = False
    _positive = positive
    if edge[0].mt == 'P':
        plemma = lemma(hg, predicate_atom(edge), same_if_none=True).root()
        if plemma in NEG_RESULTS_PREDS:
            _positive = False
        
    if contains_negator(edge[0]):
        _positive = False
    if edge[0].mt == 'B':
        if contains_negator(edge[1]):
            _positive = False
        
    _found, _ = is_positive(edge[0], keywords, _positive)
    if _found:
        found = True
    for subedge in edge[1:]:
        _found, deep_positive = is_positive(subedge, keywords, _positive)
        if _found:
            found = True
            if not deep_positive:
                _positive = False
            if not _positive:
                return True, False
    return found, _positive
    
def contains_ushape(edge):
    roots = [atom.root() for atom in edge.all_atoms()]
    if 'ushape' in roots or 'ushaped' in roots:
        return True
    return 'u' in roots

def contains_nshape(edge):
    roots = [atom.root() for atom in edge.all_atoms()]
    if 'nshape' in roots or 'nshaped' in roots:
        return True
    return 'n' in roots
    
def classify(edge):
    classes = set()
        
    keywords_found = False
    ushape = False
    nshape = False
    if contains_keywords(edge, EKC_KEYWORDS):
        keywords_found = True
    if contains_keywords(edge, CURVE_KEYWORDS):
        if contains_nshape(edge):
            nshape = True
            keywords_found = True
        elif contains_ushape(edge):
            ushape = True
            keywords_found = True
        
    if not keywords_found:
        return classes
        
    test_edge = find_test(edge)
    result_edge = find_result(edge)
    nshape = contains_nshape(edge)
    if test_edge:
        classes.add('test')
    if result_edge:
        found, positive = is_positive(result_edge, EKC_KEYWORDS | CURVE_KEYWORDS | RESULT_KEYWORDS)
        if nshape:
            if positive:
                positive = False
            else:
                found = False
        if found:
            classes.add('result')
            if positive:
                classes.add('positive')
            else:
                classes.add('negative')
    if ushape:
        classes.add('u-curve')
    if nshape:
        classes.add('n-curve')
    return classes

In [7]:
# example: look for negative results

for edge in edges:
    classes = classify(edge)
    if 'negative' in classes:
        print(hg.get_str_attribute(edge, 'text'))
        print(edge)
        print()

Fourthly, no inverted “U” shape EKC is found between the overall economic development level and the PM2.5 concentration China in 1998-2012
((fourthly/M/en (is/Mv.|f--3s-/en found/Pd.px.<pf----/en)) (no/Md/en (inverted/Ma/en (+/B.am/. u/Cm/en (+/B.am/. shape/Cc.s/en ekc/Cp.s/en)))) (between/Tt/en (in/Br.ma/en (and/J/en (the/Md/en (overall/Ma/en (+/B.am/. (economic/Ma/en development/Cc.s/en) level/Cc.s/en))) (in/Br.ma/en (the/Md/en (+/B.am/. pm2%2e5/Cc.s/en concentration/Cc.s/en)) china/Cp.s/en)) (-/B/en 1998/C#/en 2012/C#/en))))

1998–2012 Fourthly, no inverted “U” shape EKC is found between the overall economic development level and the PM2.5 concentration China in
((fourthly/M/en (no/Md/en (is/Mv.|f--3s-/en found/Pd.?px.<pf----/en))) (inverted/Ma/en u/Cm/en) (+/B.am/. shape/Cc.s/en ekc/Cp.s/en) (between/Tt/en (in/Br.ma/en (and/J/en (the/Md/en (overall/Ma/en (+/B.am/. (economic/Ma/en development/Cc.s/en) level/Cc.s/en))) (in/Br.ma/en (the/Md/en (+/B.am/. pm2%2e5/Cc.s/en concentration/C

Environmental Environmental Kuznets Kuznets Curve Curve does exist in China, but is not significant
(but/J/en ((does/Mv.|f--3s-/en exist/Pd.sx.-i-----/en) (environmental/Ma/en (+/B.am/. kuznets/Cp.s/en curve/Cp.s/en)) (in/T/en china/Cp.s/en)) ((not/Mn/en is/Pd.sc.|f--3s-/en) (environmental/Ma/en (+/B.am/. kuznets/Cp.s/en curve/Cp.s/en)) significant/Ca/en))

Using the panel data of 30 provinces and autonomous regions for the period 2006–2015, the empirical results indicate evidence for an N-shaped relationship between the EQI and income per capita; this suggests that the overall environmental quality in China would first worsen and then gradually improve as the economy continues to grow
(suggests/Pd.rsr.|f--3s-/en (indicate/P.xso.|f-----/en (using/Pd.o.|pg----/en (of/Br.ma/en (the/Md/en (+/B.am/. panel/Cc.s/en data/Cc.p/en)) (for/Br.ma/en (and/J/en (30/M#/en provinces/Cc.p/en) (autonomous/Ma/en regions/Cc.p/en)) (+/B.mm/. (the/Md/en period/Cc.s/en) 2006–2015/C#/en)))) (the/Md/en (empiri

The empirical results confirm the existence of an N-shaped relationship between economic growth and CO2 emissions in the EU-5 countries
(confirm/Pd.so.|f-----/en (the/Md/en (empirical/Ma/en results/Cc.p/en)) (of/Br.ma/en (the/Md/en existence/Cc.s/en) (in/Br.ma/en (between/Br.ma/en (an/Md/en ((n/M/en shaped/Mv.<pf----/en) relationship/Cc.s/en)) (and/J/en (economic/Ma/en growth/Cc.s/en) (+/B.am/. co2/Cc.s/en emissions/Cc.p/en))) (+/B.am/. (the/Md/en eu-5/Cp.s/en) countries/Cc.p/en))))

Our findings Notably do not endorse the validity of the EKC, albeit the majority of states support an inverted N shaped relationship
((notably/M/en (do/Mv.|f-----/en (not/Mn/en endorse/Pd.sox.-i-----/en))) (our/Mp/en findings/Cc.p/en) (of/Br.ma/en (the/Md/en validity/Cc.s/en) (the/Md/en ekc/Cp.s/en)) (albeit/T/en (support/Pd.so.|f-----/en (of/Br.ma/en (the/Md/en majority/Cc.s/en) states/Cc.p/en) (an/Md/en (inverted/Ma/en ((n/M/en shaped/Mv.<pf----/en) relationship/Cc.s/en))))))

This study further establis

In addition, the EKC hypothesis cannot be supported as the sign of coefficients on GDP and GDP2 is negative and positive, respectively
((can/Mm/en (not/Mn/en (be/Mv.-i-----/en supported/Pd.xpx.<pf----/en))) (in/T/en addition/Cc.s/en) (the/Md/en (+/B.am/. ekc/Cp.s/en hypothesis/Cc.s/en)) (as/T/en (is/Pd.sc.|f--3s-/en (of/Br.ma/en (the/Md/en sign/Cc.s/en) (on/Br.ma/en coefficients/Cc.p/en (and/J/en gdp/Cc.s/en gdp2/Cp.s/en))) (and/J/en negative/Ca/en (+/B.mm/. positive/Ca/en respectively/C/en)))))

In addition, the EKC hypothesis cannot be supported as the sign of coefficients on GDP and GDP2 is negative and positive, respectively
((can/M/en (not/Mn/en (be/Mv.-i-----/en supported/Pd.xpx.<pf----/en))) (in/T/en addition/Cc.s/en) (the/Md/en (+/B.am/. ekc/Cp.s/en hypothesis/Cc.s/en)) (as/T/en ((respectively/M/en is/Pd.sc.|f--3s-/en) (of/Br.ma/en (the/Md/en sign/Cc.s/en) (on/Br.ma/en coefficients/Cc.p/en (and/J/en gdp/Cp.s/en gdp2/Cp.s/en))) (and/J/en negative/Ca/en positive/Ca/en))))

Using 

The Autoregressive Distributed Lagged bounds approach to cointegration did not confirm the existence of EKC for any of the environmental indicators in the short run, but was robustly established in the long run for CO2 emissions and energy consumption
(but/J/en ((did/Mv.<f-----/en (not/Mn/en confirm/Pd.so.-i-----/en)) (to/Br.ma/en (the/Md/en (+/B.am/. (+/B.am/. autoregressive/Cm/en (distributed/Ma/en (lagged/Mv.<pf----/en bounds/Cc.p/en))) approach/Cc.s/en)) cointegration/Cc.s/en) (in/Br.ma/en (for/Br.ma/en (of/Br.ma/en (the/Md/en existence/Cc.s/en) ekc/Cp.s/en) (of/Br.ma/en any/Cd/en (the/Md/en (environmental/Ma/en indicators/Cc.p/en)))) (the/Md/en (short/Ma/en run/Cc.s/en)))) ((was/Mv.<f-----/en (robustly/M/en established/Pd.xx.<pf----/en)) (in/T/en (the/Md/en (long/Ma/en run/Cc.s/en))) (for/T/en (and/J/en (+/B.am/. co2/Cc.s/en emissions/Cc.p/en) (+/B.am/. energy/Cc.s/en consumption/Cc.s/en)))))

the Environmental Kuznets Curve The results indicate that the economic growth and enviro

...

However, influenced by economic and policy factors, both chemical fertilizers and pesticides showed an N-shaped curve
((however/M/en showed/Pd.xso.<f-----/en) (influenced/Pd.a.<pf----/en (by/T/en (+/B.am/. ((economic/Ma/en and/Mj/en) policy/Cc.s/en) factors/Cc.p/en))) (and/J/en (both/Md/en (+/B.am/. chemical/Cc.s/en fertilizers/Cc.p/en)) pesticides/Cc.p/en) (an/Md/en ((n/M/en shaped/Mv.<pf----/en) curve/Cc.s/en)))

By employing the dynamic ordinary least square (DOLS), and the fully modified ordinary least square FMOLS), we also find that increase in the consumption of energy and globalization increases the ecological footprint, and the EKC hypothesis is not supported for the GCC countries
(and/J/en ((also/M/en find/Pd.xsr.|f-----/en) (by/T/en (employing/P.o.|pg----/en (+/B.mm/. (and/J/en (the/Md/en (dynamic/Ma/en (ordinary/Ma/en (least/Ms/en square/Cc.s/en)))) dols/Cp.s/en) (+/B.mm/. (the/Md/en ((fully/M/en modified/Mv.<pf----/en) (ordinary/Ma/en (least/Ms/en square/Cc.s/en)))) fmols/

Result: 1) Two types of EKC model support the pfsv and pgdp present inverted N-EKC relationship, per capita forest resources along with the increase of pgdp showed the evolution process about "decline→rise→decline", but the extended EKC model fits better, and two inflection point will reach when the pgdp reached 639.79 yuan and 33 560.58 yuan and most provinces are in the ascent phase of the EKC curve
(:/J/. result/Cc.s/en (support/P.?so?.|f-----/en 1/C/en (of/Br.ma/en (two/M#/en types/Cc.p/en) (+/B.am/. ekc/Cp.s/en model/Cc.s/en)) (the/Md/en (+/B.am/. (and/J/en pfsv/Cm/en pgdp/Cc.s/en) (present/Ma/en (inverted/Ma/en (+/B.am/. (+/B.am/. n/Cp.s/en ekc/Cp.s/en) relationship/Cc.s/en))))) (but/J/en (showed/Pd.so.<f-----/en (along/Br.ma/en (per/Br.ma/en capita/C/en (+/B.am/. forest/Cc.s/en resources/Cc.p/en)) (with/Mt/en (of/Br.ma/en (the/Md/en increase/Cc.s/en) pgdp/Cc.s/en))) (about/Br.ma/en (the/Md/en (+/B.am/. evolution/Cc.s/en process/Cc.s/en)) decline→rise→decline/Cc.s/en)) (and/J/en 

This study the the environmental environmental Kuznets Kuznets curve curve Using a large sample (N = 170–174) of countries find evidence of the for SO2 emissions and income inequality has a significant impact on the turning point
(find/Pd.xso.|f-----/en (using/Pd.o.|pg----/en (+/B.mm/. (of/Br.ma/en (a/Md/en (large/Ma/en sample/Cc.s/en)) (+/B.ma/. n/Cc.s/en 170–174/C#/en)) countries/Cc.p/en)) (this/Md/en study/Cc.s/en) (and/J/en (of/Br.ma/en evidence/Cc.s/en (for/Br.ma/en (the/Md/en (the/Md/en (environmental/Ma/en (+/B.am/. kuznets/Cp.s/en curve/Cc.s/en)))) (+/B.am/. so2/Cp.s/en emissions/Cc.p/en))) (has/Pd.so.|f--3s-/en (+/B.am/. income/Cc.s/en inequality/Cc.s/en) (on/Br.ma/en (a/Md/en (significant/Ma/en impact/Cc.s/en)) (the/Md/en (+/B.am/. (the/Md/en (environmental/Ma/en (+/B.am/. kuznets/Cp.s/en curve/Cc.s/en))) (+/B.am/. turning/Cc.s/en point/Cc.s/en)))))))

For robustness checking, three estimator techniques reveal no evidence of inverted U-shape EKC consistently
((consistently/M/

The results demonstrate an N-shaped relationship, either before or after grouping, which shows a strong correlation against the effectiveness of the Environmental Kuznets Curve (EKC) hypothesis
(:/J/. (the/Md/en results/Cc.p/en) (demonstrate/P!.o??.|f-----/en (an/Md/en ((n/M/en shaped/Mv.<pf----/en) (+/B.mm/. relationship/Cc.s/en (or/J/en either/C/en before/Br/en)))) (after/T/en grouping/Cc.s/en) (shows/P.so.|f--3s-/en which/C/en (against/Br.ma/en (a/Md/en (strong/Ma/en correlation/Cc.s/en)) (of/Br.ma/en (the/Md/en effectiveness/Cc.s/en) (the/Md/en (+/B.am/. (+/B.am/. (+/B.aa/. environmental/Cm/en (+/B.aa/. kuznets/Cm/en curve/Cm/en)) ekc/Cp.s/en) hypothesis/Cc.s/en)))))))

The results demonstrate an N-shaped relationship, either before or after grouping, which shows a strong correlation against the effectiveness of the Environmental Kuznets Curve (EKC) hypothesis
(:/J/. (the/Md/en results/Cc.p/en) (demonstrate/P!.o??.|f-----/en (an/Md/en ((n/M/en shaped/Mv.<pf----/en) (+/B.mm/. relati

The results do not support the inverted U-shaped Environmental Kuznets Curve (EKC) hypothesis for per capita income (and FDI inflows) and per capita CO2 emissions in a country
((do/Mv.|f-----/en (not/Mn/en support/Pd.so.-i-----/en)) (the/Md/en results/Cc.p/en) (for/Br.ma/en (the/Md/en (inverted/Ma/en (u/Ma/en (-/Ma/en (shaped/Ma/en (+/B.am/. (+/B.aa/. (+/B.aa/. environmental/Cm/en (+/B.aa/. kuznets/Cm/en curve/Cm/en)) ekc/Cm/en) hypothesis/Cc.s/en)))))) (and/J/en (and/J/en (per/Br.ma/en capita/C/en income/Cc.s/en) (+/B.am/. fdi/Cp.s/en inflows/Cc.p/en)) (in/Br.ma/en (per/Br.ma/en capita/Cc.s/en (+/B.am/. co2/Cc.s/en emissions/Cc.p/en)) (a/Md/en country/Cc.s/en)))))

The results do not support the inverted U-shaped Environmental Kuznets Curve (EKC) hypothesis for per capita income (and FDI inflows) and per capita CO2 emissions in a country
((do/Mv.|f-----/en (not/Mn/en support/Pd.so.-i-----/en)) (the/Md/en results/Cc.p/en) (for/Br.ma/en (the/Md/en (inverted/Ma/en (u/Ma/en (-/Ma/en (shap